# Decision analysis with Julia `DecisionProgramming.jl` using Evans, 1997  
Tomás Aragón, Updated 2026-01-21, V2

I mostly replicated R `rdecision` vignette "Elementary decision tree (Evans 1997)": https://cran.r-project.org/web/packages/rdecision/vignettes/DT01-Sumatriptan.html . 

Sources:
- Briggs, Andrew Harvey, Karl Claxton, and Mark Sculpher. Decision Modelling for Health Economic Evaluation. Repr. [d. korr. Ausg. von 2007]. Handbooks in Health Economic Evaluation Series. Oxford Univ. Press, 2011.
- Evans, K. W., J. A. Boan, J. L. Evans, and A. Shuaib. “Economic Evaluation of Oral Sumatriptan Compared with Oral Caffeine/Ergotamine for Migraine.” PharmacoEconomics 12, no. 5 (1997): 565–77. https://doi.org/10.2165/00019053-199712050-00007.

This Julia Jupyter notebook was created in VS Code and contains examples from Evans, 1997. You can run this notebook in VS Code, Positron, or JupyterLab with a Julia kernel. To learn about the Julia language visit https://julialang.org/ .

We will use `DecisionProgramming.jl` Julia package: https://gamma-opt.github.io/DecisionProgramming.jl/dev/ . `DecisionProgramming.jl` is a Julia package for modeling and solving decision analysis problems using influence diagrams.

Preparation: 
- PENDING


Optional readings for background:
- Owens, Douglas K. “Analytic Tools for Public Health Decision Making.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 22, no. 5 Suppl (2002): S3-10. https://doi.org/10.1177/027298902237969.
- Owens, D. K., R. D. Shachter, and R. F. Nease. “Representation and Analysis of Medical Decision Problems with Influence Diagrams.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 241–62. https://doi.org/10.1177/0272989X9701700301.
- Nease, R. F., and D. K. Owens. “Use of Influence Diagrams to Structure Medical Decisions.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 263–75. https://doi.org/10.1177/0272989X9701700302.
- Neapolitan, Richard, Xia Jiang, Daniela P. Ladner, and Bruce Kaplan. “A Primer on Bayesian Decision Analysis With an Application to a Kidney Transplant Decision.” Transplantation 100, no. 3 (2016): 489–96. https://doi.org/10.1097/TP.0000000000001145.

## Creating the model
"This vignette is an example of modelling a decision tree using the rdecision package. It is based on the example given by Briggs (Box 2.3) which itself is based on a decision tree which compared oral Sumatriptan versus oral caffeine/Ergotamine for migraine. In this vignette, we consider the problem from the perspective of a provincial health department."

<figure>
<img src="img_Briggs2006_Box2-3.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>BOX 2.3. Decision tree from Briggs et al., 2006.</figcaption>
</figure>

### Figure 1 from Evans, 1997
Below is Figure 1 from Evans, 1997 article showing the decision tree structure. 

<figure>
<img src="img_evans1997_fig01.drawio.png" width="100%" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 1: From Evans, 1997, Figure 1.</figcaption>
</figure>

Figure 2 is the influence diagram of Figure 1. 

<figure>
<img src="img_evans1997_Influence_Diagram.drawio.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 2. Influence diagram derived from Evans, 1997, Figure 1.</figcaption>
</figure>

### Model variables
The following code defines the variables for cost, utility and effect that will be used in the model. There are 14 variables in total; 4 costs, 4 utilities and 6 probabilities.

For clarity, I coded some of the variables differently than in the original vignette. For example,

`p_norecurrence_relief_sumatriptan = 0.594` $= P(\text{no recurrence} \mid \text{relief}, \text{sumatriptan})$.

For example, 

`p_endures_norelief = 0.92` $= P(\text{endures} \mid \text{no relief})$ applies to both sumatriptan and caffeine/ergotamine treatment arms, so this variable appears in both arms.

## Set nodes, incoming edges, and states

In [1]:
using JuMP, HiGHS
using DecisionProgramming

### Set nodes with name, incoming edges, and states 

For example, consider,

`add_node!(diagram, ChanceNode("E", ["D", "R"], ["ED", "En"]))` 

- `"E"` is the name of the node
- `["D", "R"]` are the incoming edges from node D (Decision/Treatment) and node R (Relief vs No relief). This is called the "information set" of node M.
- `["ED", "En"]` are the states of node E: "ED" = Emeergency Department, "En" = Endure

I chose to use "M1" and "M0" because it is much easier to read in the code.

In [3]:
# 1. Initialize Diagram
diagram = InfluenceDiagram()

# 2. Add Nodes 
# D: D1 = Sumatriptan, D2 = Caffeine/Ergotamine
add_node!(diagram, DecisionNode("D", [], ["D1", "D2"])) 

# R: R1 = Relief, R2 = No Relief
add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R2"])) 

# N: N1 = No Recurrence, N2 = Recurrence
# Note: In the tree, recurrence implies a 2nd dose is needed
add_node!(diagram, ChanceNode("N", ["D"]["R"], ["N1", "N2"])) 

# E: E1 = ED (Emergency Dept), E2 = Endures attack
# Note: E1 represents seeking further help
add_node!(diagram, ChanceNode("E", ["R"], ["E1", "E2"])) 

# H: H1 = Hospital, H2 = Relief (at ED)
add_node!(diagram, ChanceNode("H", ["E"], ["H1", "H2"])) 

# 3. Add Value Nodes
# We add two value nodes: one for Utility (U) (QALYs) and one for Cost (C).
# They depend on all relevant chance nodes to determine the final outcome.
add_node!(diagram, ValueNode("U1", ["D", "N"])) # 2x2
add_node!(diagram, ValueNode("U2", ["D", "E"])) # 2x2
add_node!(diagram, ValueNode("U3", ["D", "E", "H"])) # 2x2x2

# Generate the internal structure
generate_arcs!(diagram)

ArgumentError: ArgumentError: invalid index: "R" of type String

### options for viewing diagram attributes  
`diagram.OPTION` where `OPTION` = `Nodes`, `Names`, `I_j`, `States`, `S`, `C`, `D`, or `V`.

In [3]:
diagram.I_j

OrderedCollections.OrderedDict{String, Vector{String}} with 8 entries:
  "D"  => []
  "R"  => ["D"]
  "N"  => ["D"]
  "E"  => ["R"]
  "H"  => ["E"]
  "U1" => ["D", "N"]
  "U2" => ["D", "E"]
  "U3" => ["D", "E", "H"]

## Assign probabilities to variables
For clarity, here is how the notation works:

REDO BELOW

- `p_E1` $=P(E = 1) = 0.20$
- `p_M1_E1_R1` $= P(M = 1 \mid E = 1, R = 1) = 0.05$
- `p_M1_E1_R0` $= P(M = 1 \mid E = 1, R = 0) = 0.33$
- `p_D1_M1` $= P(D = 1 \mid M = 1) = 0.0023$

In [ ]:
# model variables for cost
c_sumatriptan = 16.10 #
c_caffeine = 1.32 #
c_ed = 63.16 #
c_hospital = 1093.0 #

# model variables for utility
u_N1_D1 = 1.0      # A  
u_N2_D1 = 0.9      # B
u_E1_D1 = - 0.3    # C 
u_H1_E2_D1 = 0.1   # D
u_H2_E2_D1 = - 0.3 # E
u_N1_D2 = 1.0      # F  
u_N2_D2 = 0.9      # G
u_E1_D2 = - 0.3    # H
u_H1_E2_D2 = 0.1   # I
u_H2_E2_D2 = - 0.3 # J

# model variables for effect
# Sumatriptan probabilities
p_R1_D1 = 0.558
p_R2_D1 = 1 - p_R1_D1
p_N1_D1 = 0.594 
p_N2_D1 = 1 - p_N1_D1 

# Caffeine/Ergotamine probabilities
p_R1_D2 = 0.379
p_R2_D2 = 1 - p_R1_D2
p_N1_D2 = 0.703 
p_N2_D2 = 1 - p_N1_D2

# both treatment arms  
p_E1_R2 = 0.92 
p_E2_R2 = 1 - p_E1_R2
p_H1_E2 = 0.998 
p_H2_E2 = 1 - p_H1_E2

p_norecurrence_relief_caffeine = 0.703 # e11; # e12 use NA_real_ to indicate complement

p_relief_ed = .998 # e15, e17; # e16, e18 use NA_real_ to indicate complement

0.998

### Create probability matrices and assign probabilities

In [7]:
#add_node!(diagram, DecisionNode("D", [], ["S", "C"])) # Decision: Sumatriptan (S) vs Coffeine/Ergot (C)
# not applicable 

In [8]:
#add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R0"])) # R1 = relief, R0 = no relief  
X_R = ProbabilityMatrix(diagram, "R")

2×2 ProbabilityMatrix{2}:
 0.0  0.0
 0.0  0.0

In [9]:
X_R = ProbabilityMatrix(diagram, "R")
X_R["D1", "R1"] = p_R1_D1
X_R["D1", "R2"] = 1 - p_R1_D1
X_R["D2", "R1"] = p_R1_D2
X_R["D2", "R2"] = 1 - p_R1_D2
add_probabilities!(diagram, "R", X_R)

2×2 Probabilities{2}:
 0.558  0.442
 0.379  0.621

In [10]:
#add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N0"])) # N1 = non-recurrence, N0 = recurrence
X_N = ProbabilityMatrix(diagram, "N")

2×2 ProbabilityMatrix{2}:
 0.0  0.0
 0.0  0.0

In [11]:
X_N["D1", "N1"] = p_N1_D1
X_N["D1", "N2"] = 1 - p_N1_D1
X_N["D2", "N1"] = p_N1_D2
X_N["D2", "N2"] = 1 - p_N1_D2
add_probabilities!(diagram, "N", X_N)

2×2 Probabilities{2}:
 0.594  0.406
 0.703  0.297

In [12]:
#add_node!(diagram, ChanceNode("E", ["D", "R"], ["E1", "E0"])) # E1 = ED, E0 = endures
X_E = ProbabilityMatrix(diagram, "E")

2×2 ProbabilityMatrix{2}:
 0.0  0.0
 0.0  0.0

In [13]:
X_E["R1", "E1"] = p_E1_R1
X_E["R1", "E2"] = 1 - p_E1_R1
X_E["R2", "E1"] = p_E1_R2
X_E["R2", "E2"] = 1 - p_E1_R2
add_probabilities!(diagram, "E", X_E)

UndefVarError: UndefVarError: `p_E1_R1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
#add_node!(diagram, ChanceNode("H", ["D", "E"], ["H1", "H0"])) # H1 = hospital, H0 = relief
X_H = ProbabilityMatrix(diagram, "H")

In [ ]:
X_H["D1", "E1", "H1"] = p_H1_E1_D1
X_H["D1", "E1", "H2"] = 1 - p_H1_E1_D1
X_H["D1", "E2", "H1"] = p_H1_E2_D1
X_H["D1", "E2", "H2"] = 1 - p_H1_E2_D1
X_H["D2", "E1", "H1"] = p_H1_E1_D2
X_H["D2", "E1", "H2"] = 1 - p_H1_E1_D2
X_H["D2", "E2", "H1"] = p_H1_E2_D2
X_H["D2", "E2", "H2"] = 1 - p_H1_E2_D2
add_probabilities!(diagram, "H", X_H)

In [ ]:
#add_node!(diagram, ValueNode("U1", ["D", "N"])) # 2x2
Y_U1 = UtilityMatrix(diagram, "U1")

In [ ]:
Y_U1["D1", "N1"] = u_N1_D1 # A 
Y_U1["D1", "N2"] = u_N2_D1 # B
Y_U1["D2", "N1"] = u_N1_D2 # F
Y_U1["D2", "N2"] = u_N2_D2 # G
add_utilities!(diagram, "U1", Y_U1)

In [ ]:
#add_node!(diagram, ValueNode("U2", ["D", "E"])) # 2x2
Y_U2 = UtilityMatrix(diagram, "U2")

In [ ]:
Y_U2["D1", "E1"] = u_E1_D1 # C
Y_U2["D1", "E2"] = NaN
Y_U2["D2", "E1"] = u_E1_D2 # H
Y_U2["D2", "E2"] = NaN
add_utilities!(diagram, "U2", Y_U2)

In [ ]:
#add_node!(diagram, ValueNode("U3", ["D", "E", "H"])) # 2x2x2
Y_U3 = UtilityMatrix(diagram, "U3")

In [ ]:
Y_U3["D1", "E1", "H1"] = NaN
Y_U3["D1", "E1", "H2"] = NaN
Y_U3["D1", "E2", "H1"] = u_H1_E2_D1 # D
Y_U3["D1", "E2", "H2"] = u_H2_E2_D1 # E
Y_U3["D2", "E1", "H1"] = NaN
Y_U3["D2", "E1", "H2"] = NaN
Y_U3["D2", "E2", "H1"] = u_H1_E2_D2 # I
Y_U3["D2", "E2", "H2"] = u_H2_E2_D2 # J
add_utilities!(diagram, "U3", Y_U3)